In [37]:
%matplotlib inline 
import matplotlib.pyplot as plt 
import numpy as np
import os, sys
import pandas as pd
from scipy.interpolate import lagrange
from numpy.polynomial.polynomial import Polynomial
from scipy.interpolate import CubicSpline 

#Achten Sie darauf, dass Sie im header dieses Templates den Ort der Datei angeben. / 
#Make sure that you specify the location of your file in the header of this Problem Sheet.

os.chdir('C:\\Users\\your\\file\\path') #wählen Sie Ihr Verzeichnis / choose your folder

#Im Falle von / In case of:
#liefert das aktuelle Arbeitsverzeichnis, d.h. den Pfad des Verzeichnisses, unter dem das aktuelle Python-Skript läuft.
#returns the current working directory, i.e. the path of the directory under which the current Python script runs

#os.getcwd()

#Tipp: Sollten Sie Daten im Codeblock vorher oder in dem aktuellen verändern, versichern Sie sich immer, dass im nächsten Block die 
#richtigen Daten übernommen werden. Sollten Sie unsicher sein, setzen Sie den jeweiligen Block zurück, indem Sie in dem Block 
#"Strg+A" und dann "Strg+#" drücken. Führen Sie nun den Code aus, wird der Output zurückgesetzt. 
#Um die "#" zu entfernen, drücken Sie im Code erneut "Strg+A" und dann "Strg+#".

#Hint: If you should change data in the code-block before or in the current one, always make sure that in the next block the 
#correct data are transferred. If you are unsure, reset the respective block by clicking 
#"Ctrl+A" and then press "Ctrl+#" If you now execute the code, the output is reset. 
#To remove the "#" press "Ctrl+A" and then " Ctrl+#" again in the code.


<center><h1>Creating interpolation curves</h1></center>

---


<table>
    <tr style = "background: transparent;">
        <td style = "text-align: justify;">Dieses Template hilft Ihnen bei der Auswertung von Daten. Die Funktionen der kommenden Codeblocks sind wie folgt:
        <ul>
          <li>Einlesen (Mess) daten</li>
          <li>Polynomiale Interpolation + graphische Darstellung</li>
          <li>Fehlerabschätzung bei Kenntnis eines zusätzlichen Datenpunktes</li>
          <li>Splineinterpolation + graphische Darstellung</li>
        </ul>
        </td>
        <td style = "text-align: justify; vertical-align: top; width: 50%;">This template helps you to evaluate data. The functions of the upcoming code blocks are as follows:
        <ul>
          <li>read in (measurement) data</li>
          <li>polynomial interpolation + graphical representation</li>
          <li>error estimation when an additional data point is known</li>
          <li>spline interpolation + graphical representation</li>
        </ul>
        </td>  
    </tr>
</table>


<center><h3>for .csv data</h3></center>

In [70]:
#nutzen Sie die pd.read_csv oder _table()-Funktion, um die gewünschte Datei einzulesen
#use the pd.read_csv or _table() function to read the desired file

data = pd.read_csv('your_file.csv', sep=',', na_values='?');

#Schauen Sie sich an auf welche Art die Daten eingelesen wurden, um zu wissen Welche Spalten oder Zeilen Sie als arrays abspeichern.
#Look at the way the data was read to know which columns or rows you are saving as arrays.

data

<center><h3>for .xlsx data</h3></center>

In [36]:
# #Daten einlesen/ read in the data 
# #Nutzen Sie hierfür / use "pd.read_excel('...xlsx', index_col = 0)"

# data = pd.read_excel('your_file.xlsx', index_col=0)
# data

---
<center><h3>Transformation into an np array</h3></center>

In [69]:
#Um später mit den Daten das lagrange Verfahren anzuwenden, übertragen Sie die Daten in np.arrays. "...".values.tolist() und np.asarray() helfen Ihnen dabei
#To use the lagrange method with the data later, transfer the data into np.arrays. "...".values.tolist() and np.asarray() assist you in this

data_array = data.values.tolist()
npdata = np.asarray(data_array)
x = npdata[:, 'column']#Schauen Sie sich vorher die Datei an, um richtig zu extrahieren. Vorher Anführungszeichen entfernen.
y = npdata[:, 'column']#Look at the file first to extract correctly. Delete quotation marks before.

x,y

In [68]:
#Für den Fall, dass x und y den dtype = str haben. / In case x and y have the dtype = str. 

# x = np.float64(x)
# y = np.float64(y)

<center><h3>Langrange interpolation + graphical representation</h3></center>

In [67]:
poly = lagrange(x, y)

#np.round(Polynomial(poly).coef[::-1], decimals = 1)

a = Polynomial(poly).coef[::-1]

In [66]:
x_ln = np.linspace(x.min(), x.max(), 100)

P = 0

for i in range(a.size):
    
    P = P + a[i]*x_ln**i
    
plt.title('Figure 1: Langrange interpolation', loc='center')
plt.plot(x_ln, P, x, y, 'o')
plt.xlabel('x_ln', fontsize=16)
plt.ylabel('P(x)', fontsize=16);

<center><h3>Error of interpolation (in case of an additional point and an extra $x-value$)</h3></center>

In [65]:
#np.append

x_in = np.append(x, 11)

y_in = np.append(y, 4)

x_in, y_in

In [64]:
#Error of polynomial interpolation

def dividedDifferences(x_in, y_in): 

    x_in.shape #Wiedergabe der Länge des arrays in jeder Dimension. / 
    y_in.shape #Returns the length of the array in each dimension.

    if (x_in.shape != y_in.shape) or (x_in.ndim != y_in.ndim) or (x_in.size == 1) or (y_in.size == 1):

        print('Dimension und Länge (größer als eins) der arrays in jeder Dimension müssen gleich sein. / Dimension and length (bigger than one) of both arrays have to be identical')
        
    n = y_in.size #oder/or n = x_i.size / n entspricht der Länge des arrays / n comply with the length of the array
     
    pyramid = np.zeros([n, n]) #Erstellung einer nxn Matrix, gefüllt mit Nullen / production af a 5x5 matrix full of zeros
    
    pyramid[::, 0] = y_in #in die erste Spalte wird y_i eingetragen / first column of our new matrix is y_in
    
    #Die pyramid Matrix wird nun mit Werten gefüllt (entsprechend der Tabelle in 1.3.11) / The pyramid matrix is filled with values (corresponding to the table in 1.3.11)
    
    for i in range(1,n):
        
        for j in range(n-i):
            
            pyramid[j][i] = (pyramid[j+1][i-1] - pyramid[j][i-1]) / (x_in[j+i] - x_in[j]) 
    
    return pyramid #In der ersten Zeile der pyramid Matrix befinden sich die gesuchten Koeffizienten. / In the first row of the pyramid matrix you will find your Newton coefficients.

coeff = dividedDifferences(x_in, y_in)

# def newton_error(x_in, X):
    
#     n = x_in.size
#     N = np.ones(x_in.size-1) 
#     N_i = N.tolist()
    
#     for i in range(n-1): #Der neue x-Wert wird nicht mitgerechnet (siehe 1.3.16). / The further x-value is not part of the calculation. (see 1.3.16)
        
#         N_i[i] = N_i[i-1]*(X-x_in[i])
        
#     return N_i[-1]*coeff[0][-1]
# print('Error:', newton_error(x_in, 4))

#Lösung mit Rekursion

def newton_error(x_in, X, i=0, value=1):
    
    value *= (X - x_in[i])
    
    if i == x_in.size - 2: return value
    
    return newton_error(x_in, X, i=i + 1, value=value) 

print('Error:', newton_error(x_in, 4)*coeff[0][-1])

<center><h3>Spline interpolation + graphical interpolation (make sure you use the arrays without the additional point)</h3></center>

In [63]:
if (x.shape != y.shape) or (x.ndim != y.ndim) or (x.size == 1) or (y.size == 1):

        print('Dimension und Länge (größer als eins) der arrays in jeder Dimension müssen gleich sein. / Dimension and length (bigger than one) of both arrays have to be identical')
        
else: #Wichtig bei der Anwendung von CubicSpline ist, dass die Werte aufsteigend sortiert sind. / Implementation of the CubicSpline function requires an ascending order of the values.
    
        
        mask_sort = np.argsort(x)
        y_sort = y[mask_sort]
        x_sort = np.sort(x)
            
        if y_sort[0] == y_sort[-1]: print('Entferne das "#" bei ps, um die Bedingung periodic zu aktivieren. / Remove "#" in front of ps to activate the periodic condition')

        ks = CubicSpline(x_sort, y_sort, bc_type='not-a-knot')
        #ps = CubicSpline(x_sort, y_sort, bc_type='periodic') #Für die Bedingung periodic muss der erste und letzte y-Wert gleich sein. / For the periodic condition the first and last have to be the same.
        cs = CubicSpline(x_sort, y_sort, bc_type='clamped')
        ns = CubicSpline(x_sort, y_sort, bc_type='natural')

        xs = np.linspace(x_sort[0], x_sort[-1], 101)
        plt.figure(figsize=(10, 8))
        plt.plot(x_sort, y_sort, 'o', label='data')

        plt.xlabel('x_sort', fontsize=16)
        plt.ylabel('y_sort', fontsize=16)
        
        #Um das Verhalten der Ableitungen zu untersuchen, entfernen Sie den hashtag bei der jeweiligen Bedingung und tippen Sie eine Zahl, um den Grad der Ableitung zu bestimmen. Zum Beispiel verhält sich die zweite Ableitung linear, weil es sich um cubic splines handelt /
        #To analyse the behaviour of the conditions remove the hashtag below the specific condition and type a number for the order of the derivative. For example the second derivative is linear, because those are cubic splines.

        plt.plot(xs, ks(xs), label="not-a-knot")
        #plt.plot(xs, ks(xs, nu = 1), label="not-a-knot: nu-derivative")

        #plt.plot(xs, ps(xs), label="periodic")
        #plt.plot(xs, ps(xs, nu = 1), label="periodic: nu-derivative")

        plt.plot(xs, cs(xs), label="clamped")
        #plt.plot(xs, cs(xs, nu = 1), label="clamped: nu-derivative")

        plt.plot(xs, ns(xs), label="natural")
        #plt.plot(xs, ns(xs, nu = 1), label="natural: nu-derivative")

        #plt.yticks(np.arange(y_sort.min(), y_sort.max() + 0.5, 0.5))
        #plt.ylim(y_sort.min() - 0.5, y_sort.max() + 0.5)
        plt.xlim(x_sort[0] - 0.02, x_sort[-1] + 0.02)
        plt.legend(loc='lower left', ncol=2)
        plt.show()